Cross model evaluation

Given three folders with similar setup, and same dataset, eval their performance quantitavely

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

from pathlib import Path
import json
from datetime import datetime
import sys

In [2]:
%load_ext autoreload

In [3]:
from eval_metrics import SEARCH_BEHAVIOUR, METRICS, COHERENCE_MODEL_METRICS
from dataset_loader import GENRES

In [6]:
search_behaviour = SEARCH_BEHAVIOUR.GRID_SEARCH
dataset_name = GENRES.INDIE

# create the folder name for each model
training_datetime_lda = datetime(2024, 2, 16, 0, 32, 46)
training_datetime_bertopic = datetime(2024, 2, 16, 9, 47, 40)
training_datetime_bertopic_split = datetime(2024, 2, 14, 11, 15, 56)
training_datetime_ctm = datetime(2024, 2, 16, 11, 59, 10)
training_datetime_ctm_split = datetime(2024, 2, 14, 22, 4, 32)

# dataset type
dataset_type_folder = Path(f'category_{str(dataset_name)}_unique_review_text')

lda_folder = Path('lda_dev')
bertopic_folder = Path('bertopic_dev')
ctm_folder = Path('ctm_dev')

training_folder_lda = lda_folder / dataset_type_folder / f'lda_multicore_genre_{str(dataset_name)}_{search_behaviour.value}_{training_datetime_lda.strftime("%Y%m%d_%H%M%S")}'
training_folder_bertopic = bertopic_folder / dataset_type_folder / f'bertopic_genre_{str(dataset_name)}_{search_behaviour.value}_{training_datetime_bertopic.strftime("%Y%m%d_%H%M%S")}'
training_folder_bertopic_split = bertopic_folder / dataset_type_folder / f'bertopic[split]_genre_{str(dataset_name)}_{search_behaviour.value}_{training_datetime_bertopic_split.strftime("%Y%m%d_%H%M%S")}'
training_folder_ctm = ctm_folder / dataset_type_folder / f'ctm_genre_{str(dataset_name)}_{search_behaviour.value}_{training_datetime_ctm.strftime("%Y%m%d_%H%M%S")}'
training_folder_ctm_split = ctm_folder / dataset_type_folder / f'ctm[split]_genre_{str(dataset_name)}_{search_behaviour.value}_{training_datetime_ctm_split.strftime("%Y%m%d_%H%M%S")}'

# check each model folder exists
if not training_folder_lda.exists():
    print(f"Folder {training_folder_lda} does not exist")
    sys.exit(1)
if not training_folder_bertopic.exists():
    print(f"Folder {training_folder_bertopic} does not exist")
    sys.exit(1)
if not training_folder_bertopic_split.exists():
    print(f"Folder {training_folder_bertopic_split} does not exist")
    sys.exit(1)
if not training_folder_ctm.exists():
    print(f"Folder {training_folder_ctm} does not exist")
    sys.exit(1)
if not training_folder_ctm_split.exists():
    print(f"Folder {training_folder_ctm_split} does not exist")
    sys.exit(1)

In [7]:
# load the config and result files
config_dicts = {}
result_dicts = {}

for model_type, training_folder in zip(
    ['lda', 'bertopic', 'bertopic[split]', 'ctm', 'ctm[split]'], 
    [training_folder_lda, training_folder_bertopic, training_folder_bertopic_split, training_folder_ctm, training_folder_ctm_split]):
    config_file = training_folder / 'config.json'
    result_file = training_folder / 'result.json'

    with open(config_file, 'r') as f:
        config = json.load(f)
    with open(result_file, 'r') as f:
        result = json.load(f)

    config_dicts[model_type] = config
    result_dicts[model_type] = result

config_dicts

{'lda': {'model': 'lda_multicore',
  'dataset_path': 'dataset/topic_modelling/top_11_genres_unique_[review_text]/01_indie.pkl',
  'countvect_params': {'n_frequency': 70, 'ngram_range': [1, 1]},
  'lda_params': {'workers': 3,
   'chunksize': 2024,
   'passes': 5,
   'alpha': 'symmetric',
   'eta': None,
   'decay': 0.5,
   'offset': 1.0,
   'eval_every': 10,
   'iterations': 50,
   'gamma_threshold': 0.001,
   'minimum_probability': 0.01,
   'random_state': 42,
   'minimum_phi_value': 0.01,
   'per_word_topics': False,
   'dtype': "<class 'numpy.float32'>"},
  'search_space': {'lda_params': {'num_topics': [10,
     20,
     30,
     40,
     50,
     60,
     70,
     80,
     90,
     100]}},
  'metrics': ['c_npmi',
   'c_v',
   'u_mass',
   'c_uci',
   'topic_diversity',
   'inverted_rbo',
   'pairwise_jaccard_similarity'],
  'monitor': 'c_npmi',
  'search_behaviour': 'grid_search',
  'gensim_version': '4.3.2'},
 'bertopic': {'model': 'bertopic',
  'dataset_path': 'dataset/topic_model

---

View result of each model

In [9]:
# from: https://www.freecodecamp.org/news/how-to-flatten-a-dictionary-in-python-in-4-different-ways/
from collections.abc import MutableMapping

def _flatten_dict_gen(d, parent_key, sep):
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, MutableMapping):
            yield from flatten_dict(v, new_key, sep=sep).items()
        else:
            yield new_key, v


def flatten_dict(d: MutableMapping, parent_key: str = '', sep: str = '.'):
    return dict(_flatten_dict_gen(d, parent_key, sep))

def get_log_history_df(model_type):

    if model_type not in ['lda', 'bertopic', 'bertopic[split]', 'ctm', 'ctm[split]']:
        return None

    log_history = result_dicts[model_type]['log_history']


    # for each dictionary in the log_history list
    # convert them to a flattened dictionary
    # then append to a list
    log_history_flattened = [flatten_dict(log, sep='.') for log in log_history]
    log_history_flattened
    log_history_df = pd.DataFrame(log_history_flattened)

    # append a column to the dataframe for the model type
    log_history_df['model_type'] = model_type

    return log_history_df

In [20]:
# LDA log history

log_history_lda_df = get_log_history_df('lda')
log_history_lda_df

,c_npmi,c_v,u_mass,c_uci,topic_diversity,inverted_rbo,pairwise_jaccard_similarity,model_name,hyperparameters.countvect_params.ngram_range,hyperparameters.lda_params.workers,...,hyperparameters.lda_params.eval_every,hyperparameters.lda_params.iterations,hyperparameters.lda_params.gamma_threshold,hyperparameters.lda_params.minimum_probability,hyperparameters.lda_params.random_state,hyperparameters.lda_params.minimum_phi_value,hyperparameters.lda_params.per_word_topics,hyperparameters.lda_params.dtype,hyperparameters.lda_params.num_topics,model_type
0,0.035102,0.475063,-2.280489,0.253313,0.750000,0.901868,0.063821,lda_num_topics_10,"[1, 1]",3,...,10,50,0.001,0.01,42,0.01,False,<class 'numpy.float32'>,10,lda
1,0.048067,0.491005,-2.442533,0.352565,0.755000,0.946183,0.036828,lda_num_topics_20,"[1, 1]",3,...,10,50,0.001,0.01,42,0.01,False,<class 'numpy.float32'>,20,lda
2,0.046001,0.482142,-3.033115,0.308993,0.810000,0.972899,0.017635,lda_num_topics_30,"[1, 1]",3,...,10,50,0.001,0.01,42,0.01,False,<class 'numpy.float32'>,30,lda
3,0.043376,0.475536,-3.216613,0.241403,0.830000,0.987763,0.009383,lda_num_topics_40,"[1, 1]",3,...,10,50,0.001,0.01,42,0.01,False,<class 'numpy.float32'>,40,lda
4,0.033965,0.457876,-3.368236,0.106677,0.836000,0.987749,0.009416,lda_num_topics_50,"[1, 1]",3,...,10,50,0.001,0.01,42,0.01,False,<class 'numpy.float32'>,50,lda
5,0.021507,0.434359,-3.579874,-0.111672,0.863333,0.991363,0.007196,lda_num_topics_60,"[1, 1]",3,...,10,50,0.001,0.01,42,0.01,False,<class 'numpy.float32'>,60,lda
6,0.021509,0.427994,-3.542840,-0.023490,0.867143,0.991226,0.008012,lda_num_topics_70,"[1, 1]",3,...,10,50,0.001,0.01,42,0.01,False,<class 'numpy.float32'>,70,lda
7,0.017556,0.429897,-3.669534,-0.163559,0.863750,0.990302,0.008539,lda_num_topics_80,"[1, 1]",3,...,10,50,0.001,0.01,42,0.01,False,<class 'numpy.float32'>,80,lda
8,0.009322,0.417083,-3.765046,-0.301628,0.866667,0.988534,0.010298,lda_num_topics_90,"[1, 1]",3,...,10,50,0.001,0.01,42,0.01,False,<class 'numpy.float32'>,90,lda
9,0.002870,0.410964,-3.846068,-0.421206,0.875000,0.992560,0.006718,lda_num_topics_100,"[1, 1]",3,...,10,50,0.001,0.01,42,0.01,False,<class 'numpy.float32'>,100,lda


In [19]:
# bertopic log history

log_history_bertopic_df = get_log_history_df('bertopic')
log_history_bertopic_df

,c_npmi,c_v,u_mass,c_uci,topic_diversity,inverted_rbo,pairwise_jaccard_similarity,model_name,hyperparameters.sbert_params.model_name_or_path,hyperparameters.vocab_tokenizer_params.ngram_range,...,hyperparameters.umap_params.random_state,hyperparameters.hdbscan_params.min_cluster_size,hyperparameters.hdbscan_params.min_samples,hyperparameters.hdbscan_params.metric,hyperparameters.hdbscan_params.prediction_data,hyperparameters.bertopic_params.language,hyperparameters.bertopic_params.top_n_words,hyperparameters.bertopic_params.calculate_probabilities,hyperparameters.bertopic_params.nr_topics,model_type
0,0.068922,0.479882,-0.031158,0.520765,0.720000,0.909125,0.076570,bt_nr_topics_10,all-MiniLM-L6-v2,"[1, 1]",...,None,718,5,euclidean,True,english,10,True,11,bertopic
1,0.116480,0.570278,-0.067208,1.059034,0.735000,0.943201,0.056833,bt_nr_topics_20,all-MiniLM-L6-v2,"[1, 1]",...,None,718,5,euclidean,True,english,10,True,21,bertopic
2,0.106052,0.565776,-0.089398,0.827480,0.753333,0.969725,0.030222,bt_nr_topics_30,all-MiniLM-L6-v2,"[1, 1]",...,None,718,5,euclidean,True,english,10,True,31,bertopic
3,0.136357,0.616586,-0.104239,1.226877,0.762500,0.979495,0.018750,bt_nr_topics_40,all-MiniLM-L6-v2,"[1, 1]",...,None,718,5,euclidean,True,english,10,True,41,bertopic
4,0.142947,0.626157,-0.104202,1.285302,0.750000,0.984285,0.016045,bt_nr_topics_50,all-MiniLM-L6-v2,"[1, 1]",...,None,718,5,euclidean,True,english,10,True,51,bertopic
5,0.144123,0.626746,-0.125617,1.296459,0.720000,0.983383,0.015929,bt_nr_topics_60,all-MiniLM-L6-v2,"[1, 1]",...,None,718,5,euclidean,True,english,10,True,61,bertopic
6,0.147766,0.632708,-0.143367,1.355457,0.728571,0.986243,0.013817,bt_nr_topics_70,all-MiniLM-L6-v2,"[1, 1]",...,None,718,5,euclidean,True,english,10,True,71,bertopic
7,0.147684,0.638960,-0.145205,1.377332,0.695000,0.986487,0.013058,bt_nr_topics_80,all-MiniLM-L6-v2,"[1, 1]",...,None,718,5,euclidean,True,english,10,True,81,bertopic
8,0.146369,0.639427,-0.156029,1.330832,0.698889,0.987691,0.011536,bt_nr_topics_90,all-MiniLM-L6-v2,"[1, 1]",...,None,718,5,euclidean,True,english,10,True,91,bertopic
9,0.146251,0.635214,-0.156713,1.325764,0.672000,0.988042,0.011250,bt_nr_topics_100,all-MiniLM-L6-v2,"[1, 1]",...,None,718,5,euclidean,True,english,10,True,101,bertopic


In [18]:
# bertopic log history

log_history_bertopic_split_df = get_log_history_df('bertopic[split]')
log_history_bertopic_split_df

,c_npmi,c_v,u_mass,c_uci,topic_diversity,inverted_rbo,pairwise_jaccard_similarity,model_name,hyperparameters.sbert_params.model_name_or_path,hyperparameters.vocab_tokenizer_params.ngram_range,...,hyperparameters.umap_params.random_state,hyperparameters.hdbscan_params.min_cluster_size,hyperparameters.hdbscan_params.min_samples,hyperparameters.hdbscan_params.metric,hyperparameters.hdbscan_params.prediction_data,hyperparameters.bertopic_params.language,hyperparameters.bertopic_params.top_n_words,hyperparameters.bertopic_params.calculate_probabilities,hyperparameters.bertopic_params.nr_topics,model_type
0,0.087912,0.514125,-0.030990,0.712106,0.740000,0.917600,0.072987,bt_nr_topics_10,all-MiniLM-L6-v2,"[1, 1]",...,None,718,5,euclidean,True,english,10,True,11,bertopic[split]
1,0.091927,0.546044,-0.086939,0.708715,0.715000,0.950508,0.048068,bt_nr_topics_20,all-MiniLM-L6-v2,"[1, 1]",...,None,718,5,euclidean,True,english,10,True,21,bertopic[split]
2,0.132634,0.609221,-0.177171,1.142146,0.773333,0.975215,0.024384,bt_nr_topics_30,all-MiniLM-L6-v2,"[1, 1]",...,None,718,5,euclidean,True,english,10,True,31,bertopic[split]
3,0.127209,0.604627,-0.133734,1.035713,0.770000,0.983315,0.015844,bt_nr_topics_40,all-MiniLM-L6-v2,"[1, 1]",...,None,718,5,euclidean,True,english,10,True,41,bertopic[split]
4,0.132637,0.617722,-0.122820,1.141170,0.736000,0.983686,0.015047,bt_nr_topics_50,all-MiniLM-L6-v2,"[1, 1]",...,None,718,5,euclidean,True,english,10,True,51,bertopic[split]
5,0.148764,0.641973,-0.150190,1.389741,0.746667,0.986633,0.012472,bt_nr_topics_60,all-MiniLM-L6-v2,"[1, 1]",...,None,718,5,euclidean,True,english,10,True,61,bertopic[split]
6,0.148639,0.637871,-0.156927,1.333876,0.738571,0.988049,0.011273,bt_nr_topics_70,all-MiniLM-L6-v2,"[1, 1]",...,None,718,5,euclidean,True,english,10,True,71,bertopic[split]
7,0.151395,0.649797,-0.179188,1.348606,0.743750,0.989316,0.009956,bt_nr_topics_80,all-MiniLM-L6-v2,"[1, 1]",...,None,718,5,euclidean,True,english,10,True,81,bertopic[split]
8,0.143635,0.635756,-0.235601,1.242396,0.713333,0.988531,0.010506,bt_nr_topics_90,all-MiniLM-L6-v2,"[1, 1]",...,None,718,5,euclidean,True,english,10,True,91,bertopic[split]
9,0.154432,0.652349,-0.211077,1.385180,0.713000,0.989739,0.009544,bt_nr_topics_100,all-MiniLM-L6-v2,"[1, 1]",...,None,718,5,euclidean,True,english,10,True,101,bertopic[split]


In [17]:
# CTM log history

log_history_ctm_df = get_log_history_df('ctm')
log_history_ctm_df

,c_npmi,c_v,u_mass,c_uci,topic_diversity,inverted_rbo,pairwise_jaccard_similarity,model_name,hyperparameters.sbert_params.model_name_or_path,hyperparameters.countvect_params.max_features,...,hyperparameters.ctm_params.hidden_sizes,hyperparameters.ctm_params.dropout,hyperparameters.ctm_params.lr,hyperparameters.ctm_params.momentum,hyperparameters.ctm_params.solver,hyperparameters.ctm_params.num_epochs,hyperparameters.ctm_params.n_components,hyperparameters.ctm_params.bow_size,hyperparameters.ctm_params.contextual_size,model_type
0,-0.020440,0.437020,-0.002036,-1.081756,0.900000,0.973171,0.013385,ctm_n_components_10,all-MiniLM-L6-v2,2000,...,"[100, 100]",0.2,0.002,0.99,adam,25,10,2000,384,ctm
1,0.007691,0.469173,-0.035353,-0.589697,0.870000,0.990286,0.008906,ctm_n_components_20,all-MiniLM-L6-v2,2000,...,"[100, 100]",0.2,0.002,0.99,adam,25,20,2000,384,ctm
2,-0.000925,0.475408,-0.050994,-0.849703,0.813333,0.986428,0.010001,ctm_n_components_30,all-MiniLM-L6-v2,2000,...,"[100, 100]",0.2,0.002,0.99,adam,25,30,2000,384,ctm
3,0.008792,0.475750,-0.074545,-0.647426,0.672500,0.978846,0.017752,ctm_n_components_40,all-MiniLM-L6-v2,2000,...,"[100, 100]",0.2,0.002,0.99,adam,25,40,2000,384,ctm
4,0.009563,0.479744,-0.073833,-0.591397,0.644000,0.978085,0.017431,ctm_n_components_50,all-MiniLM-L6-v2,2000,...,"[100, 100]",0.2,0.002,0.99,adam,25,50,2000,384,ctm
5,0.009982,0.470651,-0.075321,-0.501629,0.545000,0.973242,0.021218,ctm_n_components_60,all-MiniLM-L6-v2,2000,...,"[100, 100]",0.2,0.002,0.99,adam,25,60,2000,384,ctm
6,0.005604,0.462807,-0.078927,-0.591050,0.488571,0.965547,0.025716,ctm_n_components_70,all-MiniLM-L6-v2,2000,...,"[100, 100]",0.2,0.002,0.99,adam,25,70,2000,384,ctm
7,-0.003982,0.442942,-0.113644,-0.796546,0.417500,0.960690,0.027750,ctm_n_components_80,all-MiniLM-L6-v2,2000,...,"[100, 100]",0.2,0.002,0.99,adam,25,80,2000,384,ctm
8,0.008654,0.463077,-0.084067,-0.481113,0.384444,0.957794,0.030310,ctm_n_components_90,all-MiniLM-L6-v2,2000,...,"[100, 100]",0.2,0.002,0.99,adam,25,90,2000,384,ctm
9,-0.000856,0.445437,-0.095970,-0.689628,0.354000,0.951050,0.033675,ctm_n_components_100,all-MiniLM-L6-v2,2000,...,"[100, 100]",0.2,0.002,0.99,adam,25,100,2000,384,ctm


In [24]:
# CTM log history

log_history_ctm_split_df = get_log_history_df('ctm[split]')
log_history_ctm_split_df

,c_npmi,c_v,u_mass,c_uci,topic_diversity,inverted_rbo,pairwise_jaccard_similarity,model_name,hyperparameters.sbert_params.model_name_or_path,hyperparameters.countvect_params.max_features,...,hyperparameters.ctm_params.hidden_sizes,hyperparameters.ctm_params.dropout,hyperparameters.ctm_params.lr,hyperparameters.ctm_params.momentum,hyperparameters.ctm_params.solver,hyperparameters.ctm_params.num_epochs,hyperparameters.ctm_params.n_components,hyperparameters.ctm_params.bow_size,hyperparameters.ctm_params.contextual_size,model_type
0,-0.061572,0.433881,-0.008918,-2.177961,0.860000,0.964184,0.029741,ctm_n_components_10,all-MiniLM-L6-v2,2000,...,"[100, 100]",0.2,0.002,0.99,adam,25,10,2000,384,ctm[split]
1,-0.053110,0.449271,-0.039072,-2.132967,0.870000,0.986387,0.013554,ctm_n_components_20,all-MiniLM-L6-v2,2000,...,"[100, 100]",0.2,0.002,0.99,adam,25,20,2000,384,ctm[split]
2,-0.028067,0.469380,-0.031157,-1.595666,0.823333,0.986616,0.011732,ctm_n_components_30,all-MiniLM-L6-v2,2000,...,"[100, 100]",0.2,0.002,0.99,adam,25,30,2000,384,ctm[split]
3,-0.027873,0.483478,-0.066113,-1.693331,0.805000,0.988228,0.009897,ctm_n_components_40,all-MiniLM-L6-v2,2000,...,"[100, 100]",0.2,0.002,0.99,adam,25,40,2000,384,ctm[split]
4,-0.025110,0.487836,-0.094659,-1.596580,0.712000,0.984306,0.012545,ctm_n_components_50,all-MiniLM-L6-v2,2000,...,"[100, 100]",0.2,0.002,0.99,adam,25,50,2000,384,ctm[split]
5,-0.011628,0.483797,-0.108311,-1.210940,0.660000,0.980974,0.014722,ctm_n_components_60,all-MiniLM-L6-v2,2000,...,"[100, 100]",0.2,0.002,0.99,adam,25,60,2000,384,ctm[split]
6,-0.022762,0.466958,-0.113866,-1.434976,0.607143,0.978907,0.015995,ctm_n_components_70,all-MiniLM-L6-v2,2000,...,"[100, 100]",0.2,0.002,0.99,adam,25,70,2000,384,ctm[split]
7,-0.024701,0.450321,-0.138700,-1.419862,0.536250,0.976569,0.017526,ctm_n_components_80,all-MiniLM-L6-v2,2000,...,"[100, 100]",0.2,0.002,0.99,adam,25,80,2000,384,ctm[split]
8,-0.006518,0.476038,-0.116936,-0.961876,0.494444,0.971178,0.022035,ctm_n_components_90,all-MiniLM-L6-v2,2000,...,"[100, 100]",0.2,0.002,0.99,adam,25,90,2000,384,ctm[split]
9,-0.013009,0.466502,-0.113916,-1.136932,0.434000,0.970011,0.023040,ctm_n_components_100,all-MiniLM-L6-v2,2000,...,"[100, 100]",0.2,0.002,0.99,adam,25,100,2000,384,ctm[split]


Then group all the models, keeping only the model name and the metric

use the model name as the key to view hyperparameter of the model

In [26]:
metrics_names = [m.value for m in METRICS]

interested_columns = metrics_names + ['model_type', 'model_name']

# concat the three log history dataframe with only the interested columns
log_history_df = pd.concat(
    [log_history_lda_df, log_history_bertopic_df, log_history_bertopic_split_df, log_history_ctm_df, log_history_ctm_split_df]
)[interested_columns]

log_history_df

,u_mass,c_v,c_uci,c_npmi,topic_diversity,inverted_rbo,pairwise_jaccard_similarity,model_type,model_name
0,-2.280489,0.475063,0.253313,0.035102,0.750000,0.901868,0.063821,lda,lda_num_topics_10
1,-2.442533,0.491005,0.352565,0.048067,0.755000,0.946183,0.036828,lda,lda_num_topics_20
2,-3.033115,0.482142,0.308993,0.046001,0.810000,0.972899,0.017635,lda,lda_num_topics_30
3,-3.216613,0.475536,0.241403,0.043376,0.830000,0.987763,0.009383,lda,lda_num_topics_40
4,-3.368236,0.457876,0.106677,0.033965,0.836000,0.987749,0.009416,lda,lda_num_topics_50
5,-3.579874,0.434359,-0.111672,0.021507,0.863333,0.991363,0.007196,lda,lda_num_topics_60
6,-3.542840,0.427994,-0.023490,0.021509,0.867143,0.991226,0.008012,lda,lda_num_topics_70
7,-3.669534,0.429897,-0.163559,0.017556,0.863750,0.990302,0.008539,lda,lda_num_topics_80
8,-3.765046,0.417083,-0.301628,0.009322,0.866667,0.988534,0.010298,lda,lda_num_topics_90
9,-3.846068,0.410964,-0.421206,0.002870,0.875000,0.992560,0.006718,lda,lda_num_topics_100


In [27]:
# with that information, we can sort by different metrics

# sort by c_v
log_history_df.sort_values(by='c_v', ascending=False)

,u_mass,c_v,c_uci,c_npmi,topic_diversity,inverted_rbo,pairwise_jaccard_similarity,model_type,model_name
9,-0.211077,0.652349,1.385180,0.154432,0.713000,0.989739,0.009544,bertopic[split],bt_nr_topics_100
7,-0.179188,0.649797,1.348606,0.151395,0.743750,0.989316,0.009956,bertopic[split],bt_nr_topics_80
5,-0.150190,0.641973,1.389741,0.148764,0.746667,0.986633,0.012472,bertopic[split],bt_nr_topics_60
8,-0.156029,0.639427,1.330832,0.146369,0.698889,0.987691,0.011536,bertopic,bt_nr_topics_90
7,-0.145205,0.638960,1.377332,0.147684,0.695000,0.986487,0.013058,bertopic,bt_nr_topics_80
6,-0.156927,0.637871,1.333876,0.148639,0.738571,0.988049,0.011273,bertopic[split],bt_nr_topics_70
8,-0.235601,0.635756,1.242396,0.143635,0.713333,0.988531,0.010506,bertopic[split],bt_nr_topics_90
9,-0.156713,0.635214,1.325764,0.146251,0.672000,0.988042,0.011250,bertopic,bt_nr_topics_100
6,-0.143367,0.632708,1.355457,0.147766,0.728571,0.986243,0.013817,bertopic,bt_nr_topics_70
5,-0.125617,0.626746,1.296459,0.144123,0.720000,0.983383,0.015929,bertopic,bt_nr_topics_60


In [28]:
# sort by c_npmi

log_history_df.sort_values(by='c_npmi', ascending=False)

,u_mass,c_v,c_uci,c_npmi,topic_diversity,inverted_rbo,pairwise_jaccard_similarity,model_type,model_name
9,-0.211077,0.652349,1.385180,0.154432,0.713000,0.989739,0.009544,bertopic[split],bt_nr_topics_100
7,-0.179188,0.649797,1.348606,0.151395,0.743750,0.989316,0.009956,bertopic[split],bt_nr_topics_80
5,-0.150190,0.641973,1.389741,0.148764,0.746667,0.986633,0.012472,bertopic[split],bt_nr_topics_60
6,-0.156927,0.637871,1.333876,0.148639,0.738571,0.988049,0.011273,bertopic[split],bt_nr_topics_70
6,-0.143367,0.632708,1.355457,0.147766,0.728571,0.986243,0.013817,bertopic,bt_nr_topics_70
7,-0.145205,0.638960,1.377332,0.147684,0.695000,0.986487,0.013058,bertopic,bt_nr_topics_80
8,-0.156029,0.639427,1.330832,0.146369,0.698889,0.987691,0.011536,bertopic,bt_nr_topics_90
9,-0.156713,0.635214,1.325764,0.146251,0.672000,0.988042,0.011250,bertopic,bt_nr_topics_100
5,-0.125617,0.626746,1.296459,0.144123,0.720000,0.983383,0.015929,bertopic,bt_nr_topics_60
8,-0.235601,0.635756,1.242396,0.143635,0.713333,0.988531,0.010506,bertopic[split],bt_nr_topics_90


In [29]:
# sort by inverted rbo (an indicator of diversity), The higher the better

log_history_df.sort_values(by='inverted_rbo', ascending=False)

,u_mass,c_v,c_uci,c_npmi,topic_diversity,inverted_rbo,pairwise_jaccard_similarity,model_type,model_name
9,-3.846068,0.410964,-0.421206,0.002870,0.875000,0.992560,0.006718,lda,lda_num_topics_100
5,-3.579874,0.434359,-0.111672,0.021507,0.863333,0.991363,0.007196,lda,lda_num_topics_60
6,-3.542840,0.427994,-0.023490,0.021509,0.867143,0.991226,0.008012,lda,lda_num_topics_70
7,-3.669534,0.429897,-0.163559,0.017556,0.863750,0.990302,0.008539,lda,lda_num_topics_80
1,-0.035353,0.469173,-0.589697,0.007691,0.870000,0.990286,0.008906,ctm,ctm_n_components_20
9,-0.211077,0.652349,1.385180,0.154432,0.713000,0.989739,0.009544,bertopic[split],bt_nr_topics_100
7,-0.179188,0.649797,1.348606,0.151395,0.743750,0.989316,0.009956,bertopic[split],bt_nr_topics_80
8,-3.765046,0.417083,-0.301628,0.009322,0.866667,0.988534,0.010298,lda,lda_num_topics_90
8,-0.235601,0.635756,1.242396,0.143635,0.713333,0.988531,0.010506,bertopic[split],bt_nr_topics_90
3,-0.066113,0.483478,-1.693331,-0.027873,0.805000,0.988228,0.009897,ctm[split],ctm_n_components_40


In [30]:
# sort by jaccard simularity, the higher the better (an indicator of similarity)
# (an inverse of diversity, hence not that useful)

log_history_df.sort_values(by='pairwise_jaccard_similarity', ascending=False)

,u_mass,c_v,c_uci,c_npmi,topic_diversity,inverted_rbo,pairwise_jaccard_similarity,model_type,model_name
0,-0.031158,0.479882,0.520765,0.068922,0.720000,0.909125,0.076570,bertopic,bt_nr_topics_10
0,-0.030990,0.514125,0.712106,0.087912,0.740000,0.917600,0.072987,bertopic[split],bt_nr_topics_10
0,-2.280489,0.475063,0.253313,0.035102,0.750000,0.901868,0.063821,lda,lda_num_topics_10
1,-0.067208,0.570278,1.059034,0.116480,0.735000,0.943201,0.056833,bertopic,bt_nr_topics_20
1,-0.086939,0.546044,0.708715,0.091927,0.715000,0.950508,0.048068,bertopic[split],bt_nr_topics_20
1,-2.442533,0.491005,0.352565,0.048067,0.755000,0.946183,0.036828,lda,lda_num_topics_20
9,-0.095970,0.445437,-0.689628,-0.000856,0.354000,0.951050,0.033675,ctm,ctm_n_components_100
8,-0.084067,0.463077,-0.481113,0.008654,0.384444,0.957794,0.030310,ctm,ctm_n_components_90
2,-0.089398,0.565776,0.827480,0.106052,0.753333,0.969725,0.030222,bertopic,bt_nr_topics_30
0,-0.008918,0.433881,-2.177961,-0.061572,0.860000,0.964184,0.029741,ctm[split],ctm_n_components_10
